In [5]:
import numpy as np
import pandas as pd

In [6]:
columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
#df = pd.read_csv('WISDM_at_v2.0_raw.txt', header = None, names = columns)
#df['z-axis'] = df['z-axis'].str.rstrip(';')

In [13]:
df = pd.read_csv('users_below_id_600.csv')
df = df[df['user'] != 579]
df = df[df.columns.unique()].replace(';', '')
df = df.dropna()


for i in df['user'].unique():
    if len(df[df['user'] == i]) < 5000:
        df = df[df['user'] != i]
 

df['timestamp'] = df['timestamp'].astype(int)
df['x-axis'] = df['x-axis'].astype(float)
df['y-axis'] = df['y-axis'].astype(float)
df['z-axis'] = df['z-axis'].astype(float)



In [18]:
for user in df['user'].unique():
    for activity in df['activity'].unique():
        user_activity_to_process = df[df['user'] == user][df[df['user'] == user]['activity'] == activity]
        if len(user_activity_to_process['timestamp']) == 0:
            df[df['user'] == user][df[df['user'] == user]['activity'] != activity] = df[df['user'] == user][df[df['user'] == user]['activity'] != activity]
            break
        df[df['user'] == user][df[df['user'] == user]['activity'] == activity]['timestamp'] -= user_activity_to_process['timestamp'].iloc[0]
        df[df['user'] == user][df[df['user'] == user]['activity'] == activity]['timestamp'] /= 1000

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
# standardize accelerometer data 

# min max scaling

# standard scaler 

# perform windowing of the data for each user, and each activity

# translate each minute window into an individual row with a label (which is the user)

# add bias for each window (3 biases for each window) + bias so it will end up being a 4 x 1200 
# feature vector. Maybe 

# add row to matrix of all data for a particular activity

# decide on model, and train model to see what we can do